## Identify the missing ones : 

> All the depos in 77 phages : 

In [2]:
import os 
import pandas as pd
from collections import Counter

path_classic="/media/concha-eloko/Linux/77_strains_phage_project"

to_discard = ["K80PH1317a"]

rbp_df = pd.read_csv(f"{path_classic}/RBP_77_phages.0907.csv", sep = "\t", header =0).fillna("None")
rbp_df = rbp_df[~rbp_df["Phage"].isin(to_discard)]
depo_df = rbp_df[rbp_df["Function"] == "depolymerase"]


In [3]:
depos_format_all = []
for _, row in depo_df.iterrows() :
    cds_name = "_".join(row["CDS"].split(" "))
    depo_name = f'{row["Phage"]}__{cds_name}'
    depos_format_all.append(depo_name)


In [4]:
path_ppt = "/media/concha-eloko/Linux/PPT_clean"

current_depo = pd.read_csv(f"{path_ppt}/77_current_dpos.tsv", names = ["depos"])
depo_current_format = current_depo["depos"].tolist()

In [5]:
missing_ones = set(depos_format_all) - set(depo_current_format)
len(missing_ones)

11

In [6]:
missing_ones

{'K18PH07C1__cds_231',
 'K23PH08C2__cds_220',
 'K23PH08C2__cds_225',
 'K23PH08C2__cds_228',
 'K45PH128C2__cds_227',
 'K45PH128C2__cds_232',
 'K45PH128C2__cds_235',
 'K54lambda1_1_1__cds_225',
 'K54lambda1_1_1__cds_230',
 'K54lambda1_1_1__cds_233',
 'K65PH164__cds_197'}

***
## Create a multi fasta file : 

In [13]:
path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_annotation = f"{path_project}/annotation_classic"

missing_depo_df = pd.read_csv(f"{path_project}/missing_dpos_delination.0907.tsv", names = ["depos", "boundaries"], sep = "\t")


In [14]:
missing_depo_df

,depos,boundaries
0,K18PH07C1__cds_231,175-476
1,K23PH08C2__cds_220,167-472
2,K23PH08C2__cds_225,107-256
3,K23PH08C2__cds_228,86-161
4,K45PH128C2__cds_227,167-472
5,K45PH128C2__cds_232,107-274
6,K45PH128C2__cds_235,85-155
7,K54lambda1_1_1__cds_225,169-476
8,K54lambda1_1_1__cds_230,108-272
9,K54lambda1_1_1__cds_233,87-151


In [15]:
with open(f"{path_ppt}/missing_depo.09072024.fasta", "w") as outfile :
    for _, row in missing_depo_df.iterrows() :
        if row["boundaries"] != "ignore" :
            phage = row["depos"].split("__")[0]
            cds = row["depos"].split("_")[-1]
            faa_file = open(f"{path_classic}/annotation_29092023/{phage}/{phage}.multi.annotated.faa").read().split(">")
            if row["boundaries"] == "full" :
                seq = faa_file[int(cds)].split('\n')[1]
            else :
                start , end = int(row["boundaries"].split("-")[0]) , int(row["boundaries"].split("-")[1])
                seq = faa_file[int(cds)].split('\n')[1][start : end]
            outfile.write(f">{phage}__cds_{cds}\n{seq}\n\n")


> Fix missing some missing dpo

In [16]:
from Bio import SeqIO
path_ppt = "/media/concha-eloko/Linux/PPT_clean"

missing_depo_seqio = {record.description : record.seq for record in SeqIO.parse(f"{path_ppt}/missing_depo.09072024.fasta", "fasta")}
len(missing_depo_seqio)

10

> Move it to the server :

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/missing_depo.09072024.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project/missing_depo.09072024.fasta.esm2_out \
/media/concha-eloko/Linux/77_strains_phage_project/


In [17]:
import torch
import os 
import pandas as pd
from tqdm import tqdm 


path_esm = "/media/concha-eloko/Linux/77_strains_phage_project/missing_depo.09072024.fasta.esm2_out"

embeddings_esm = {}
for file in tqdm(os.listdir(path_esm)) :
    path_file = f"{path_esm}/{file}"
    index = file.split(".")[0]
    embb = torch.load(f"{path_file}")["mean_representations"][33].tolist()
    embeddings_esm[index] = embb
    
with open(f"/media/concha-eloko/Linux/77_strains_phage_project/missed_dpo.77_phages.domains.09072024.esm2.embedding.csv" , "w") as outfile :
    for index in tqdm(embeddings_esm) :
        outfile.write(f"{index},")
        #for _,  emb in enumerate(embeddings_esm[index]) :
        #    outfile.write(f"{emb},")
        data_emb = ",".join([str(item) for item in embeddings_esm[index]])
        outfile.write(f"{data_emb}\n")
        #outfile.write("\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1681.89it/s]
